# Employee Data

- employee_id
- first_name
- last_name
- phone_num
    - country_code
    - area_code
    - exchange_code
    - subscriber_number
- email_id
- address
    - street_name
    - unit_number
    - city
    - county
    - state
    - zip_code
    - extended_zip_code
        - sector
        - segment

In [1]:
# Import required modules/libraries
from pyspark.sql.types import (StructType, StructField, StringType, IntegerType)
from pyspark.sql import SparkSession, Row
from datetime import date, datetime
from pyspark.sql.functions import col, struct
import random

In [2]:
# Create a spark connection
spark = SparkSession.builder.appName("Employee Analysis").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/03 15:34:20 WARN Utils: Your hostname, Pulastyas-Mac-mini.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.133 instead (on interface en1)
26/01/03 15:34:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/03 15:34:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Employee Schema

extended_zip_code_schema = StructType([
    StructField("sector", IntegerType()),  # Example: 54
    StructField("segment", IntegerType()) # Example: 01
])

address_schema = StructType([
    StructField("street_name", StringType(), False),
    StructField("unit_number", StringType()),
    StructField("city", StringType(), False),
    StructField("county", StringType()),
    StructField("state", StringType(), False),
    StructField("zip_code", IntegerType(), False),
    StructField("extended_zip_code", extended_zip_code_schema) # Nesting the previous structure
])

phone_num_schema = StructType([                                 # e.g., 1 (555)-123-4567
    StructField("country_code", IntegerType(), False),          # e.g., "1"
    StructField("area_code", IntegerType(), False),             # e.g., "555"
    StructField("exchange_code", IntegerType(), False),         # e.g., "123"
    StructField("subscriber_number", IntegerType(), False)      # e.g., "4567"
])

employee_schema = StructType([
    StructField("employee_id", IntegerType(), False),
    StructField("dept_id", IntegerType(), False),    
    StructField("first_name", StringType(), False),
    StructField("middle_name", StringType()),
    StructField("last_name", StringType(), False),
    StructField("phone_num", phone_num_schema, False),         # Nested Phone Structure
    StructField("email_id", StringType(), False),
    StructField("address", address_schema, False)              # Nested Address Structure
])

In [4]:
# Department Schema

dept_schema = StructType([
    StructField("dept_id", IntegerType(), False),
    StructField("dept_name", StringType(), False),
    StructField("dept_head_emp_id", IntegerType(), False),
    StructField("loc_address", address_schema, False)
])

In [5]:
# Generate synthetic data

FIRST_NAMES = ["Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah", "Ivy", "Jack"]
LAST_NAMES = ["Smith", "Jones", "Williams", "Brown", "Davis", "Miller", "Wilson", "Moore", "Taylor", "Anderson"]
CITIES = ["New York", "Chicago", "Boston", "Seattle", "Austin", "Denver"]
STATES = ["NY", "IL", "MA", "WA", "TX", "CO"]
DEPARTMENTS = [
    (10, "Engineering"),
    (20, "Sales"),
    (30, "HR"),
    (40, "Finance"),
    (50, "Marketing"),
]

In [6]:
# Create a list of department IDs that we will assign to employees
DEPT_IDS = [id for id, name in DEPARTMENTS]

In [7]:
def generate_address(city, state):
    """Generates a nested address Row consistent with address_schema."""
    zip_code_base = random.randint(10000, 99999)
    return Row(
        street_name=f"{random.randint(100, 999)} {random.choice(['Oak', 'Pine', 'Main'])} St",
        unit_number=random.choice([None, str(random.randint(1, 200))]),
        city=city,
        county=f"{city} County",
        state=state,
        zip_code=zip_code_base,
        extended_zip_code=Row(
            sector=random.randint(10, 99),
            segment=random.randint(10, 99)
        )
    )

In [8]:
dept_data = []
# Ensure each department gets a location
for dept_id, dept_name in DEPARTMENTS:
    city = random.choice(CITIES)
    state = STATES[CITIES.index(city)]
    
    # Generate a unique employee ID for the department head (ensuring it's not the same as a future employee ID)
    dept_head_emp_id = 1000 + dept_id
    
    dept_data.append(Row(
        dept_id=dept_id,
        dept_name=dept_name,
        dept_head_emp_id=dept_head_emp_id,
        loc_address=generate_address(city, state)
    ))

In [9]:
print(dept_data)

[Row(dept_id=10, dept_name='Engineering', dept_head_emp_id=1010, loc_address=Row(street_name='194 Main St', unit_number=None, city='Denver', county='Denver County', state='CO', zip_code=88583, extended_zip_code=Row(sector=34, segment=47))), Row(dept_id=20, dept_name='Sales', dept_head_emp_id=1020, loc_address=Row(street_name='110 Pine St', unit_number=None, city='Chicago', county='Chicago County', state='IL', zip_code=33026, extended_zip_code=Row(sector=41, segment=52))), Row(dept_id=30, dept_name='HR', dept_head_emp_id=1030, loc_address=Row(street_name='292 Oak St', unit_number=None, city='Denver', county='Denver County', state='CO', zip_code=70296, extended_zip_code=Row(sector=71, segment=85))), Row(dept_id=40, dept_name='Finance', dept_head_emp_id=1040, loc_address=Row(street_name='953 Main St', unit_number=None, city='New York', county='New York County', state='NY', zip_code=69799, extended_zip_code=Row(sector=83, segment=67))), Row(dept_id=50, dept_name='Marketing', dept_head_emp_

In [10]:
# Create a test data
employee_data = []
for i in range(1, 101): # 100 employees
    emp_id = 1000 + i
    dept_id = random.choice(DEPT_IDS) # Ensure dept_id is valid
    
    first = random.choice(FIRST_NAMES)
    last = random.choice(LAST_NAMES)
    
    city = random.choice(CITIES)
    state = STATES[CITIES.index(city)]
    
    employee_data.append(Row(
        employee_id=emp_id,
        dept_id=dept_id,
        first_name=first,
        middle_name=random.choice([None, "Xavier", "Yancy"]), # Allow for null middle_name
        last_name=last,
        
        # Nested Phone Structure (Non-Nullable fields generated)
        phone_num=Row(
            country_code=1,
            area_code=random.randint(200, 999),
            exchange_code=random.randint(100, 999),
            subscriber_number=random.randint(1000, 9999)
        ),
        
        email_id=f"{first.lower()}.{last.lower()}@company.com",
        
        # Nested Address Structure (Non-Nullable fields generated)
        address=generate_address(city, state)
    ))

print(employee_data)

[Row(employee_id=1001, dept_id=20, first_name='George', middle_name='Yancy', last_name='Miller', phone_num=Row(country_code=1, area_code=372, exchange_code=354, subscriber_number=8180), email_id='george.miller@company.com', address=Row(street_name='917 Main St', unit_number=None, city='Denver', county='Denver County', state='CO', zip_code=50588, extended_zip_code=Row(sector=97, segment=69))), Row(employee_id=1002, dept_id=40, first_name='George', middle_name='Yancy', last_name='Jones', phone_num=Row(country_code=1, area_code=633, exchange_code=241, subscriber_number=7056), email_id='george.jones@company.com', address=Row(street_name='413 Oak St', unit_number=None, city='Denver', county='Denver County', state='CO', zip_code=18464, extended_zip_code=Row(sector=88, segment=23))), Row(employee_id=1003, dept_id=20, first_name='Ethan', middle_name='Xavier', last_name='Moore', phone_num=Row(country_code=1, area_code=693, exchange_code=491, subscriber_number=2351), email_id='ethan.moore@compan

In [11]:
df_employee = spark.createDataFrame(employee_data, schema=employee_schema)

df_employee.show()

+-----------+-------+----------+-----------+---------+-------------------+--------------------+--------------------+
|employee_id|dept_id|first_name|middle_name|last_name|          phone_num|            email_id|             address|
+-----------+-------+----------+-----------+---------+-------------------+--------------------+--------------------+
|       1001|     20|    George|      Yancy|   Miller|{1, 372, 354, 8180}|george.miller@com...|{917 Main St, NUL...|
|       1002|     40|    George|      Yancy|    Jones|{1, 633, 241, 7056}|george.jones@comp...|{413 Oak St, NULL...|
|       1003|     20|     Ethan|     Xavier|    Moore|{1, 693, 491, 2351}|ethan.moore@compa...|{657 Oak St, NULL...|
|       1004|     40|    George|      Yancy|    Moore|{1, 670, 407, 1043}|george.moore@comp...|{983 Oak St, NULL...|
|       1005|     20|       Bob|       NULL|   Taylor|{1, 310, 492, 9657}|bob.taylor@compan...|{438 Main St, NUL...|
|       1006|     10|    George|      Yancy|    Brown|{1, 769, 7

In [12]:
df_department = spark.createDataFrame(dept_data, schema=dept_schema)

df_department.show()

+-------+-----------+----------------+--------------------+
|dept_id|  dept_name|dept_head_emp_id|         loc_address|
+-------+-----------+----------------+--------------------+
|     10|Engineering|            1010|{194 Main St, NUL...|
|     20|      Sales|            1020|{110 Pine St, NUL...|
|     30|         HR|            1030|{292 Oak St, NULL...|
|     40|    Finance|            1040|{953 Main St, NUL...|
|     50|  Marketing|            1050|{905 Pine St, 1, ...|
+-------+-----------+----------------+--------------------+



In [13]:
df_employee.filter(col("employee_id") == 1010).show()

+-----------+-------+----------+-----------+---------+-------------------+--------------------+--------------------+
|employee_id|dept_id|first_name|middle_name|last_name|          phone_num|            email_id|             address|
+-----------+-------+----------+-----------+---------+-------------------+--------------------+--------------------+
|       1010|     10|     Diana|       NULL|    Smith|{1, 531, 257, 7558}|diana.smith@compa...|{794 Main St, NUL...|
+-----------+-------+----------+-----------+---------+-------------------+--------------------+--------------------+



In [14]:
df_employee.alias("e").join(df_department.alias("d"), on=(col("e.employee_id") == col("d.dept_head_emp_id")), how="inner").select(col("e.employee_id")).show()

+-----------+
|employee_id|
+-----------+
|       1010|
|       1020|
|       1030|
|       1040|
|       1050|
+-----------+



In [15]:
spark.stop()